In [207]:

import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt

from sympy.solvers import solve
from sympy import Symbol
import math
import random
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from numpy import linalg as LA

import time

In [208]:
def function(x,E,theta,eta,w,delta,alpha,s_0,b,rho,tv):

    lambda_ = alpha*(x[4]+s_0)**b
    
    
    A = x[2]*x[3]/w
    mu = 25
    
    #dot_W = x[0]*rho*((mu/x[2]-1)+float(np.random.normal(0,0.22,1)))
    dot_W = x[0]*rho*((mu/x[2]-1))
    #dot_W = 0
    dot_L = theta*(lambda_*(A-x[1])-A)
    
    #dot_L = theta*(lambda_*(E*lambda_-x[1])-E*lambda_ )
    
    dot_p = (x[2]* (1-x[3]) * dot_W / x[0] + eta*(w-x[0])*(E-A+x[1])+ w*dot_L ) / (1-x[3]*w-(1-x[3])*x[0])
    #dot_p = ( eta*(w-x[0])*(E-A+x[1])+ w*dot_L ) 
    
    
    # / (1-x[3]*w-(1-x[3])*x[0])
    dot_n = x[3]* (-(1-w)*dot_p/x[2] + 1/A*(eta*(E-A+x[1])+dot_L))
    #dot_n = 0
    
    #dot_s = -delta*(x[4]) + delta * (np.log(1-dot_p/x[2]))**2
    #dot_s = -delta*(x[4]) + delta * (np.log(1-dot_p/x[2]))**2
    dot_s = -delta*(x[4]) + delta * tv**2*(-dot_p/x[2])**2
    
    
    return([dot_W, dot_L, dot_p, dot_n, dot_s])
   

In [209]:
def sys(iv, dt, time,function,E,theta,eta,w,delta,alpha,s_0,b,rho,tv):
    n = int(time/dt)
    t = np.linspace(0,time,n)
    x = np.array([[ i for i in iv]]*n)
    n_ord = len(iv)
    
    a = iv
    
    for i in range(1,n):
        X = function(a,E,theta,eta,w,delta,alpha,s_0,b,rho,tv)
        X[0] = X[0] 
        a = a + np.array(X)*dt
        x[i] = a
    return np.array(x)


In [368]:
rho= 0.1
#rho = 100

E = 2
mu = 25
theta = 10
eta = 10
w = 0.5


#tv = 10**-1
#alpha = 10**-2
#s_0 = 10**-6


tv = 10**-1
alpha = 0.01
s_0 = 10**-6


b = -0.5

In [369]:
T = 300
dt = 10**-2
delta = 0.5

start_time = time.time()

p0 = 20
L0 = 20
x= sys([0.2,L0,p0,0.5,1], dt,T,function,E,theta,eta,w,delta,alpha,s_0,b,rho,tv)

print("--- %s seconds ---" % (time.time() - start_time))


n = int(T/dt)
t = np.linspace(0,T,n)

--- 0.4133946895599365 seconds ---


In [370]:
fig = plt.figure(figsize=(5,5))
fig.subplots_adjust(wspace = 0.5, hspace = 0.3)
ax1 = fig.add_subplot(5,1,1)
ax2 = fig.add_subplot(5,1,2)
ax3 = fig.add_subplot(5,1,3)
ax4 = fig.add_subplot(5,1,4)
ax5 = fig.add_subplot(5,1,5)

n = int(T/dt)
t = np.linspace(0,T,n)

#lambda_ = alpha * (x[:,4] + s_0)**b
w = 0.5
#A = x[:,2]*x[:,3]/w

T1 = 0
T2 = 20000
#xx= np.array([x[500*i] for i in range(int(len(x)//1000))])
#t = np.array([t[500*i] for i in range(int(len(t)//1000))])
xx = x
lambda_ = alpha * (xx[:,4] + s_0)**b

tau = 1
#ax1.plot(t,10*(A-x[:,1])/x[:,2], 'r-',label ='W')
ax1.plot(tau*t,xx[:,0], 'k-',label ='W')
ax1.set_ylabel(r'$W$')


ax2.plot(t,(1-1/lambda_)*x[:,2]*x[:,3]/w, 'k',label ='L',alpha = 0.2)
ax2.plot(tau*t,xx[:,1], 'k-',label ='L')
#ax2.plot(t,x[:,1]-(lambda_-1)*E, 'b-',label ='L')
ax2.set_ylabel(r'$L$')



###### AX3 P ######
#num = (1-n)*rho*mu+eta*(w-W)*(E+L)-w*theta*lambda_*L
#denom = (1-n)*rho + eta*(w-W)*n/w -theta*n*(lambda_-1)
num = (1-n)*rho*mu+eta*(w-W)*(E)
denom = (1-n)*rho - eta*(w-W)*n/ (w* lambda_)

A = x[:,2]*x[:,3]/w
mu = 25
#dot_W = x[0]*rho*((mu/x[2]-1)+float(np.random.normal(0,0.22,1)))
dot_W = x[:,0]*rho*((mu/x[:,2]-1))
#dot_W = 0
dot_L = theta*(lambda_*(A-x[:,1])-A)
ax3.plot(t,(x[:,2]* (1-x[:,3]) * dot_W / x[:,0] + eta*(w-x[:,0])*(E-A+x[:,1])+ w*dot_L ) / (1-x[:,3]*w-(1-x[:,3])*x[:,0]))
#ax3.plot(t,num/denom,'k',alpha=0.5)
#ax3.plot(tau*t,xx[:,2], 'k-',label ='p')

#ax3.set_ylabel(r'$p$')


###### AX4 ######

ax4.plot(tau*t,xx[:,3], 'k-',label ='n')
#ax4.plot(t,w/x[:,2] * (E+x[:,1]), 'r--',alpha=0.4)
ax4.set_ylabel(r'$n$')


#ax5.plot(t,A/E, 'k--',label ='s',alpha=0.5)
ax5.plot(tau*t,lambda_, 'k-',label ='s')

#ax5.plot(t,(np.log(1-theta*lambda_*(E*(lambda_-1)-x[:,1]) / (x[:,1]+10)))**2, 'g--',label ='s')
ax5.set_ylabel(r'$ \lambda $')
ax5.set_xlabel(r'$ t $')

ax2.set_ylim([-10,20])
#ax3.set_ylim([-10,40])
plt.grid()
plt.show()


/home/lcbob/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


In [485]:
#num = (1-n)*rho*mu+eta*(w-W)*(E+L)-w*theta*lambda_*L
#denom = (1-n)*rho + eta*(w-W)*n/w -theta*n*(lambda_-1)
num = (1-n)*rho*mu+eta*(w-W)*(E)
denom = (1-n)*rho - eta*(w-W)*n/ (w* lambda_)

A = x[:,2]*x[:,3]/w
mu = 25
#dot_W = x[0]*rho*((mu/x[2]-1)+float(np.random.normal(0,0.22,1)))
dot_W = x[:,0]*rho*((mu/x[:,2]-1))
#dot_W = 0
dot_L = theta*(lambda_*(A-x[:,1])-A)
#ax3.plot(t,num/denom,'k',alpha=0.5)
#ax3.plot(tau*t,xx[:,2], 'k-',label ='p')
dot_p = (x[:,2]* (1-x[:,3]) * dot_W / x[:,0] + eta*(w-x[:,0])*(E-A+x[:,1])+ w*dot_L ) / (1-x[:,3]*w-(1-x[:,3])*x[:,0])
dot_n = x[:,3]* (-(1-w)*dot_p/x[:,2] + 1/A*(eta*(E-A+x[:,1])+dot_L))

plt.figure()
T1,T2 = 3600,6332
T1,T2 = 3500,6332
plt.plot(t[T1:T2],L[T1:T2])
plt.plot(t[T1:T2],A[T1:T2]*(1-1/lambda_[T1:T2]))

plt.plot(t[T1:T2],p[T1:T2])
#num = (1-n)*rho*mu + eta*(w-W)*(E+L) - w *theta*lambda_*L
#dem =  (1-n)*rho + eta*n/w * (w-W) + theta*n*(lambda_-1)

num = (1-n)*rho*mu + eta*(w-W)*E
dem =  (1-n)*rho + eta*n/(w*lambda_) *(w-W) 
plt.plot(t[T1:T2],num[T1:T2]/dem[T1:T2])

#plt.plot(t[T1:T2],a0[T1:T2])
#plt.plot(t,dot_n / x[:,3])
#plt.plot(t[T1:T2],a1[T1:T2])
#plt.plot(t[T1:T2],theta*w*A[T1:T2]/ (p[T1:T2] * lambda_[T1:T2] * (1-n[T1:T2]*w-(1-n[T1:T2])*W[T1:T2])))

#plt.plot(t,theta*x[:,3]/lambda_ /(1-x[:,3]*w-(1-x[:,3])*x[:,0]))
plt.plot(t[T1:T2],lambda_[T1:T2]/10)



/home/lcbob/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


In [455]:
plt.plot(p)

In [ ]:
plt.plot()

## Slow Manifold

In [314]:
np.sqrt(2/3)

0.816496580927726

##### - Implicit function countour method

In [541]:
def plot_implicit(fn,tv,s_0,alpha, bbox=(-0.5,1)):
    ''' create a plot of an implicit function
    fn  ...implicit function (plot where fn==0)
    bbox ..the x,y,and z limits of plotted interval'''
    xmin, xmax, ymin, ymax, zmin, zmax = bbox*3
    ymin,ymax= 0, 1
    zmin, zmax = 0,12
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    A = np.linspace(xmin, xmax, 1000) # resolution of the contour
    B = np.linspace(ymin, ymax, 100) # number of slices
    C = np.linspace(0, zmax, 100)
    A1,A2 = np.meshgrid(A,C) # grid on which the contour is plotted

    

    for y in B: # plot contours in the XZ plane
        X,Z = A1,A2
        Y = fn(X,y,Z,tv,s_0,alpha)
        cset = ax.contour(X, Y+y, Z, [y], zdir='y',alpha=0.2)


    # must set plot limits because the contour will likely extend
    # way beyond the displayed level.  Otherwise matplotlib extends the plot limits
    # to encompass all values in the contour.
    ax.set_zlim3d(zmin,zmax)
    ax.set_xlim3d(xmin,xmax)
    ax.set_ylim3d(ymin,ymax)

    plt.show()

In [544]:
def goursat_tangle(x,y,z,tv,s_0,alpha):
    #t,s_0,alpha = 1,10**-2,1
    return tv**2*(y*z-x)**2*z**2+s_0*z**2-alpha**2
    #return tv**2*(y*z-x)**2+s_0-alpha**2/z**2

#tv,s_0,alpha = 1,0.01,1
tv,s_0,alpha =  0.1, 10**-6, 0.01

print('tv,s_0,alpha = ',tv,s_0,alpha)
plot_implicit(goursat_tangle,tv,s_0,alpha)

tv,s_0,alpha =  0.1 1e-06 0.01


/home/lcbob/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


###### - a0, a1, lambda_ def

In [380]:
W,L,p,n,s = np.transpose(x)
A = n*p/w

a1 = w/p * theta * (A - L) / (1-n*w-(1-n)*W)
a0 = (theta*n-(1-n)*rho*(mu/p-1)-eta/p*(w-W)*(E-A+L)) / (1-n*w-(1-n)*W)

tv,s_0,alpha =  0.1, 10**-6, 0.01
lambda_ = alpha * (s + s_0)**b

In [487]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
T1,T2 = 3600,6332

plt.plot(a0[T1:T2],p[T1:T2],lambda_[T1:T2],'.',markersize = 0.4)
#ax.set_xlim3d(0,1)

/home/lcbob/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


In [543]:
plt.figure()

#T1,T2 = 3610,4100
T1,T2 = 3600,4500

plt.plot(10*a0[T1:T2],a1[T1:T2],'+--')
plt.plot(0*a0[T1:T2],a1[T1:T2],'+--')

/home/lcbob/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


In [545]:
T1,T2 = 3610,4100 #10
#T1,T2 = 2280,4100 #100
#T1,T2 = 3600,5000 #1


plt.plot(a0[T1:T2],a1[T1:T2],lambda_[T1:T2],'+--')
plt.plot(a0[0:1],a1[0:1],lambda_[0:1],'g.')
plt.xlabel('a0')

Text(-0.034953059062034934, -0.09081071126310042, 'a0')

In [283]:
T1,T2 = 2000,10000

plt.plot(a0[T1:T2],a1[T1:T2],lambda_[T1:T2],'.')

#plt.plot(a0[0:1],a1[0:1],lambda_[0:1],'g.')
plt.xlabel('a0')

Text(-0.03495305906203494, -0.09081071126310043, 'a0')

In [377]:

fig = plt.figure(figsize=(5,5))

ax1 = fig.add_subplot(3,1,1)
ax2 = fig.add_subplot(3,1,2)
ax3 = fig.add_subplot(3,1,3)

ax1.plot(t,lambda_)
ax2.plot(t,tv**2*(a1*lambda_ - a0)**2 + s_0 - (lambda_/alpha)**-2)
ax3.plot(t,a0)


/home/lcbob/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


In [165]:
plt.plot(theta*n / (1-n*w-(1-n)*W), theta*n/lambda_ /(1-n*w-(1-n)*W) ,lambda_)

In [166]:
plt.plot(theta*n / (1-n*w-(1-n)*W),w/p * theta*(A-L) /(1-n*w-(1-n)*W) ,lambda_)

In [81]:
plt.plot(theta*n / (1-n*w-(1-n)*W),a1,lambda_)

In [210]:
plt.figure()
dot_L = theta*(lambda_*(A-L)-A)
#plt.plot(t,dot_L)
plt.plot(t,a1)
plt.plot(t,w/p * theta*(E) /(1-n*w-(1-n)*W),alpha=0.5)

In [90]:
plt.figure()
plt.plot(t,(1-1/lambda_)*A,alpha=0.5)
plt.plot(t,L,alpha=0.5)

### Compare $\dot{n}$ and $\dot{n}_{slow }$

In [526]:
a_ = (1-w)*rho*(mu/p-1)+ eta*(1-w)/(w*lambda_)
b_ = -((1-w)*(rho*(mu/p-1)+E*eta/lambda_) + eta/lambda_)
c_ = eta*w*E / p

In [527]:
DELTA = b_**2 - 4*a_*c_
D_ = [i*(i>0) for i in DELTA]
d_ = np.sqrt(D_)

In [532]:
dot_L = theta*(lambda_*(A-L)-A)
dot_p = ((1-n)*rho*(mu-p)+eta*(w-W)*(E-A+L)+w*dot_L) / (1-n*w-(1-n)*W)
df3dn =  ( -rho*(mu-p)-(w-W)*eta*p/w + w*theta*(lambda_-1)*p/w ) / (1-w*n-(1-n)*W) - (-w+W)*((1-n)*rho*(mu-p)+eta*(E-A+L)+w*dot_L) / (1-w*n-(1-n)*W)**2

df4dn = w/p*(-eta*p/w+theta*(lambda_-1)*p/w)

dfndn = -(1-w)/p *(dot_p + n*df3dn) + df4dn

In [540]:
plt.figure()
T1,T2 = 3600,6332
T1,T2 = 3500,6632
plt.plot(t[T1:T2],n[T1:T2]-(-b_[T1:T2] - d_[T1:T2] )/ (2*a_[T1:T2]) )
#plt.plot(t[T1:T2],(-b_[T1:T2] + d_[T1:T2] )/ (2*a_[T1:T2]))
#plt.plot(t[T1:T2],(-b_[T1:T2] - d_[T1:T2] )/ (2*a_[T1:T2]))
#plt.plot(t[T1:T2],(dfndn[T1:T2]>0))
#plt.plot(t[T1:T2],lambda_[T1:T2]/10)



/home/lcbob/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


In [11]:
lambda_ = alpha*(x[:,4]+s_0)**b


A = x[:,2]*x[:,3]/w
mu = 25

#dot_W = x[0]*rho*((mu/x[2]-1)+float(np.random.normal(0,0.22,1)))
dot_W = x[:,0]*rho*((mu/x[:,2]-1))
#dot_W = 0
dot_L = theta*(lambda_*(A-x[:,1])-A)
dot_p = (x[:,2]* (1-x[:,3]) * dot_W / x[:,0] + eta*(w-x[:,0])*(E-A+x[:,1])+ w*dot_L ) / (1-x[:,3]*w-(1-x[:,3])*x[:,0])
dot_n = x[:,3]* (-(1-w)*dot_p/x[:,2] + 1/A*(eta*(E-A+x[:,1])+dot_L))

In [20]:

plt.figure()



plt.plot(t,dot_n)
plt.plot(t,eta*(E*w/x[:,2] - x[:,3]/lambda_),alpha=0.3)
#plt.plot(t,w/x[:,2]*(eta*(E-A+x[:,1])),alpha=0.5)
#plt.plot(t,-x[:,3]*(1-w)*dot_p/x[:,2] + w/x[:,2]*(eta*(E-A+x[:,1])),alpha=0.2)

# Eigenvalues

In [108]:
def eigenvalue(value):

    #parameter
    theta,eta,s0,alpha,bb,mu,rho,w,tv = [10,10,10**-6,0.01,-0.5,25,0.1,0.5,0.1]
       #theta,eta,s0,alpha,bb,mu,rho,w,tv = [10,10,0,0.01,-0.5,25,0.1,0.5,0.1]

    #value
    W,L,p,n,s = value
    
    Mat = np.zeros((5,5))
    A = n*p / w
    lambda_ = alpha*(s+s0)**bb
    
    Mat[0,0] = rho*(mu/p-1)
    Mat[0,2] = -W*rho*mu/p**2

    Mat[1,1] = -theta*lambda_
    Mat[1,2] = theta*(lambda_-1)*n/w
    Mat[1,3] = theta*(lambda_-1)*p/w
    Mat[1,4] = theta*(A-L)

    dot_L = theta*(lambda_*(A-L)-A)
    dot_p = ((1-n)*rho*(mu-p)+eta*(w-W)*(E-A+L)+w*dot_L) / (1-n*w-(1-n)*W)

    #df3dW = -eta*(E-A+L) / (1-w*n-(1-n)*W) - (1-n)*dot_p / (1-n*w-(1-n)*W)**2
    df3dW = -eta*(E-A+L) / (1-w*n-(1-n)*W) + (1-n)*dot_p / (1-n*w-(1-n)*W)**2 # change sign
    df3dl = ( (w-W)*eta + w*Mat[1,1] ) / (1-w*n-(1-n)*W)
    df3dp =  ( (1-n)*(-rho)-(w-W)*eta*n/w + w*Mat[1,2] ) / (1-w*n-(1-n)*W)
    df3dn =  ( -rho*(mu-p)-(w-W)*eta*p/w + w*Mat[1,3] ) / (1-w*n-(1-n)*W) - (-w+W)*((1-n)*rho*(mu-p)+eta*(E-A+L)+w*dot_L) / (1-w*n-(1-n)*W)**2
    df3dlambda = w*(theta*(A-L)) / (1-w*n-(1-n)*W)

    Mat[2,0] = df3dW
    Mat[2,1] = df3dl
    Mat[2,2] = df3dp
    Mat[2,3] = df3dn
    Mat[2,4] = df3dlambda

    
    df4dl = w/p*(eta+Mat[1,1])
    df4dp = -w/p**2*(eta*(E-A+L)+theta*(lambda_*(A-L)-A)) + w/p * (eta*(-n/w)+Mat[1,2])
    df4dn = w/p*(-eta*p/w+Mat[1,3])
    df4dlambda = w/p*theta*(A-L)

    Mat[3,0] = -(1-w)*n/p * df3dW
    Mat[3,1] = -(1-w)*n/p * df3dl + df4dl
    Mat[3,2] = -(1-w)*n * (df3dp/p-dot_p/p**2) + df4dp
    Mat[3,3] = -(1-w)/p *(dot_p + n*df3dn) + df4dn
    Mat[3,4] = -(1-w)*n/p * df3dlambda + df4dlambda


    a0 = ((1-n)*rho*(mu-p)*eta*(w-W)*(E-A+L)+w*theta*(-A)) / (p*(1-n*w-(1-n)*W))
    a1 = w*theta*(A-L) / (p*(1-n*w-(1-n)*W))



    #Mat[4,0] = delta / (2*alpha**2) * tv**2 *lambda_**2*2*dot_p/p * df3dW /p
    #Mat[4,1] = delta / (2*alpha**2) * tv**2 *lambda_**2*2*dot_p/p * df3dl /p
    #Mat[4,2] = delta / (2*alpha**2) * tv**2 *lambda_**2*2*dot_p/p * (df3dp /p - dot_p/p**2)
    #Mat[4,3] = delta / (2*alpha**2) * tv**2 *lambda_**2*2*dot_p/p * (df3dn /p)
    #Mat[4,4] = delta / (2*alpha**2) * (alpha**2 - 3*s0*lambda_**2 + tv**2*lambda_**2*(3*a0**2+2*4*a0*a1*lambda_+5*a1**2*lambda_**2))

    
    Mat[4,0] = - delta / (2*alpha**2) * tv**2 *lambda_**3*2*dot_p/p * df3dW /p
    Mat[4,1] = - delta / (2*alpha**2) * tv**2 *lambda_**3*2*dot_p/p * df3dl /p
    Mat[4,2] = - delta / (2*alpha**2) * tv**2 *lambda_**3*2*dot_p/p * (df3dp /p - dot_p/p**2)
    Mat[4,3] = - delta / (2*alpha**2) * tv**2 *lambda_**3*2*dot_p/p * (df3dn /p)
    Mat[4,4] = delta / (2*alpha**2) * (alpha**2 - 3*s0*lambda_**2 - tv**2*lambda_**2*(3*a0**2+2*4*a0*a1*lambda_+5*a1**2*lambda_**2))

    
    eigva, v = LA.eig(Mat)
    return(eigva,v)

In [109]:
list_values = np.array([eigenvalue(value)[0] for value in x])

In [110]:
T1 = 0
T2 = 20000

In [113]:
fig = plt.figure(figsize=(8,8))


for i in range(5):
    print(i)
    fig.subplots_adjust(wspace = 0.5, hspace = 0.3)
    ax1 = fig.add_subplot(3,1,1)
    ax2 = fig.add_subplot(3,1,2)
    ax3 = fig.add_subplot(3,1,3)
    
    ax1.plot(t[T1:T2],lambda_[T1:T2], 'g-',label ='s',markersize=0.1)
        
    ax2.plot(t[T1:T2],list_values[:,i][T1:T2].real,'+--',markersize=1,alpha=0.5)
    #ax2.plot(tt[T1:T2],0+0*list_values[:,i][T1:T2].real,'k--',alpha=0.5)
    #ax2.plot(t[T1:T2],-1+0*list_values[:,i][T1:T2].real,'k--',alpha=0.5)
    #ax2.plot(t[T1:T2],np.absolute(list_values[:,i][T1:T2]),'-',alpha=0.5)
    
    ax3.plot(t[T1:T2],list_values[:,i][T1:T2].imag,'.',markersize=0.5)

ax1.set_ylabel('$\lambda$')        
ax2.set_ylabel('$Re(\lambda_i)$') 
ax2.grid()
ax3.set_ylabel('$Imag(\lambda_i)$')         
ax3.set_xlabel('$t$')     

plt.show()

0
1
2
3
4


/home/lcbob/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  import sys
/home/lcbob/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  
/home/lcbob/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reus

In [94]:
w,v = eigenvalue(x[int(4400)])
for i in range(len(w)):
    print(w[i],'\n',v[:,i])

(-1443.6341311155654+0j) 
 [ 1.43131325e-08+0.j  1.37914482e-02+0.j  1.49896789e-02+0.j
  1.58147333e-04+0.j -9.99792519e-01+0.j]
(-0.027784875930387865+0j) 
 [-2.01896105e-01+0.j  2.67529421e-02+0.j -9.79006213e-01+0.j
  8.30770658e-03+0.j -2.45567411e-04+0.j]
(-0.16776687487607184+0j) 
 [ 0.00938521+0.j -0.05174751+0.j  0.99857544+0.j -0.00891731+0.j
  0.00129984+0.j]
(-0.8074117844486703+9.355455623339182j) 
 [9.11114210e-06+1.08403509e-04j 6.69526493e-01-8.24439417e-05j
 7.40921607e-01+0.00000000e+00j 5.99966101e-03+1.63104132e-03j
 4.13181070e-03-5.20931935e-02j]
(-0.8074117844486703-9.355455623339182j) 
 [9.11114210e-06-1.08403509e-04j 6.69526493e-01+8.24439417e-05j
 7.40921607e-01-0.00000000e+00j 5.99966101e-03-1.63104132e-03j
 4.13181070e-03+5.20931935e-02j]


In [69]:

w,v = eigenvalue(x[int(4600)])
for i in range(len(w)):
    print(w[i],'\n',v[:,i])

(-1768.836024113208+0j) 
 [ 8.61863919e-09+0.j  1.12252058e-02+0.j  1.17329442e-02+0.j
  1.23465761e-04+0.j -9.99868150e-01+0.j]
(-0.024205070708052445+0j) 
 [-4.65674921e-01+0.j  2.75081720e-02+0.j -8.84488283e-01+0.j
  8.40475803e-03+0.j -7.55655175e-05+0.j]
(-0.15588333332227028+0j) 
 [-9.66579548e-03+0.j  6.27149374e-02+0.j -9.97934048e-01+0.j
  1.00391238e-02+0.j -5.10265199e-04+0.j]
(1.8630228107480566+9.707474118854746j) 
 [ 1.81495845e-05-9.34796056e-05j -6.88598319e-01+4.39347930e-03j
 -7.24735408e-01+0.00000000e+00j -5.87589574e-03-1.28711076e-03j
  4.48918218e-03+2.27001474e-02j]
(1.8630228107480566-9.707474118854746j) 
 [ 1.81495845e-05+9.34796056e-05j -6.88598319e-01-4.39347930e-03j
 -7.24735408e-01-0.00000000e+00j -5.87589574e-03+1.28711076e-03j
  4.48918218e-03-2.27001474e-02j]


In [153]:
w,v = eigenvalue(x[int(5200)])
for i in range(len(w)):
    print(w[i],'\n',v[:,i])

(1.3974620053231668+0j) 
 [ 0.00161994+0.j -0.61484522+0.j -0.70680871+0.j -0.0126482 +0.j
 -0.34960577+0.j]
(-0.13208228823802243+0.11995165605954951j) 
 [ 0.01304953+0.01098273j -0.02116042+0.01166899j  0.99948615+0.j
 -0.00182579+0.00026544j -0.01081844+0.0056713j ]
(-0.13208228823802243-0.11995165605954951j) 
 [ 0.01304953-0.01098273j -0.02116042-0.01166899j  0.99948615-0.j
 -0.00182579-0.00026544j -0.01081844-0.0056713j ]
(-12.960293507410576+0j) 
 [ 1.98036470e-04+0.j  5.84939297e-01+0.j  8.08037050e-01+0.j
  1.44577714e-02+0.j -6.86518665e-02+0.j]
(-9.520600193788134+0j) 
 [ 2.98810135e-04+0.j  4.38168242e-01+0.j  8.95895217e-01+0.j
 -3.74730271e-03+0.j -7.32544877e-02+0.j]


In [157]:
w,v = eigenvalue(x[int(6000)])
for i in range(len(w)):
    print(w[i],'\n',v[:,i])

(-391.5259875209356+0j) 
 [-2.88379663e-07+0.j -5.15156115e-02+0.j -6.45937644e-02+0.j
 -7.06846037e-04+0.j  9.96580798e-01+0.j]
(-0.04169128255840321+0j) 
 [-0.06789646+0.j  0.01135046+0.j -0.99761319+0.j  0.00528723+0.j
 -0.00109418+0.j]
(-0.20996397296329244+0j) 
 [ 0.00901037+0.j -0.01450788+0.j  0.99983041+0.j -0.00539597+0.j
  0.0042851 +0.j]
(-5.980263945971549+5.484236898446388j) 
 [1.26058559e-04+0.00011591j 5.94665576e-01+0.02345995j
 7.93832090e-01+0.j         6.19934037e-03+0.00282767j
 9.03769877e-02-0.08624755j]
(-5.980263945971549-5.484236898446388j) 
 [1.26058559e-04-0.00011591j 5.94665576e-01-0.02345995j
 7.93832090e-01-0.j         6.19934037e-03-0.00282767j
 9.03769877e-02+0.08624755j]


## fast eigenvalues

In [184]:
def fast_eigenvalue(value):

    #parameter
    theta,eta,s0,alpha,bb,mu,rho,w,tv = [10,10,10**-6,0.01,-0.5,25,0.1,0.5,0.1]
       #theta,eta,s0,alpha,bb,mu,rho,w,tv = [10,10,0,0.01,-0.5,25,0.1,0.5,0.1]

    #value
    W,L,p,n,s = value
    
    Mat = np.zeros((3,3))
    A = n*p / w
    lambda_ = alpha*(s+s0)**bb
    

    Mat[0,0] = -theta*lambda_
    Mat[0,1] = theta*(lambda_-1)*n/w
    Mat[0,2] = theta*(A-L)

    dot_L = theta*(lambda_*(A-L)-A)
    dot_p = ((1-n)*rho*(mu-p)+eta*(w-W)*(E-A+L)+w*dot_L) / (1-n*w-(1-n)*W)

    #df3dW = -eta*(E-A+L) / (1-w*n-(1-n)*W) - (1-n)*dot_p / (1-n*w-(1-n)*W)**2
    df3dW = -eta*(E-A+L) / (1-w*n-(1-n)*W) + (1-n)*dot_p / (1-n*w-(1-n)*W)**2 # change sign
    df3dl = ( (w-W)*eta + w*Mat[1,1] ) / (1-w*n-(1-n)*W)
    df3dp =  ( (1-n)*(-rho)-(w-W)*eta*n/w + w*Mat[0,1] ) / (1-w*n-(1-n)*W)
    df3dlambda = w*(theta*(A-L)) / (1-w*n-(1-n)*W)

    Mat[1,0] = df3dl
    Mat[1,1] = df3dp
    Mat[1,2] = df3dlambda

    
    df4dl = w/p*(eta+Mat[0,0])
    df4dp = -w/p**2*(eta*(E-A+L)+theta*(lambda_*(A-L)-A)) + w/p * (eta*(-n/w)+Mat[0,1])
    df4dlambda = w/p*theta*(A-L)

    a0 = ((1-n)*rho*(mu-p)*eta*(w-W)*(E-A+L)+w*theta*(-A)) / (p*(1-n*w-(1-n)*W))
    a1 = w*theta*(A-L) / (p*(1-n*w-(1-n)*W))



    #Mat[4,0] = delta / (2*alpha**2) * tv**2 *lambda_**2*2*dot_p/p * df3dW /p
    #Mat[4,1] = delta / (2*alpha**2) * tv**2 *lambda_**2*2*dot_p/p * df3dl /p
    #Mat[4,2] = delta / (2*alpha**2) * tv**2 *lambda_**2*2*dot_p/p * (df3dp /p - dot_p/p**2)
    #Mat[4,3] = delta / (2*alpha**2) * tv**2 *lambda_**2*2*dot_p/p * (df3dn /p)
    #Mat[4,4] = delta / (2*alpha**2) * (alpha**2 - 3*s0*lambda_**2 + tv**2*lambda_**2*(3*a0**2+2*4*a0*a1*lambda_+5*a1**2*lambda_**2))

    
    Mat[2,0] = - delta / (2*alpha**2) * tv**2 *lambda_**3*2*dot_p/p * df3dl /p
    Mat[2,1] = - delta / (2*alpha**2) * tv**2 *lambda_**3*2*dot_p/p * (df3dp /p - dot_p/p**2)
    Mat[2,2] = delta / (2*alpha**2) * (alpha**2 - 3*s0*lambda_**2 - tv**2*lambda_**2*(3*a0**2+2*4*a0*a1*lambda_+5*a1**2*lambda_**2))

    
    eigva, v = LA.eig(Mat)
    return(eigva,v)

In [185]:
list_values = np.array([fast_eigenvalue(value)[0] for value in x])

In [186]:
fig = plt.figure(figsize=(8,8))

T1 = 0
T2 = 20000

for i in range(3):
    print(i)
    fig.subplots_adjust(wspace = 0.5, hspace = 0.3)
    ax1 = fig.add_subplot(3,1,1)
    ax2 = fig.add_subplot(3,1,2)
    ax3 = fig.add_subplot(3,1,3)
    
    ax1.plot(t[T1:T2],lambda_[T1:T2], 'g-',label ='s',markersize=0.1)
        
    ax2.plot(t[T1:T2],list_values[:,i][T1:T2].real,'+--',markersize=1,alpha=0.5)
    #ax2.plot(tt[T1:T2],0+0*list_values[:,i][T1:T2].real,'k--',alpha=0.5)
    #ax2.plot(t[T1:T2],-1+0*list_values[:,i][T1:T2].real,'k--',alpha=0.5)
    #ax2.plot(t[T1:T2],np.absolute(list_values[:,i][T1:T2]),'-',alpha=0.5)
    
    ax3.plot(t[T1:T2],list_values[:,i][T1:T2].imag,'.',markersize=0.5)

ax1.set_ylabel('$\lambda$')        
ax2.set_ylabel('$Re(\lambda_i)$') 
ax2.grid()
ax3.set_ylabel('$Imag(\lambda_i)$')         
ax3.set_xlabel('$t$')     

plt.show()

0
1
2


/home/lcbob/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  if __name__ == '__main__':
/home/lcbob/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  # Remove the CWD from sys.path while we load stuff.
/home/lcbob/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: MatplotlibDeprecationWarning: Addi

In [204]:
w,v = fast_eigenvalue(x[int(3620)])
for i in range(len(w)):
    print(w[i],'\n',v[:,i])

-1025.8665861412558 
 [ 0.01818889  0.01427371 -0.99973268]
-8.692051613757886 
 [-0.90286107  0.4298292  -0.0094205 ]
-2.2308237009128664 
 [ 0.04833987 -0.99878526 -0.00955258]


In [205]:
w,v = fast_eigenvalue(x[int(3621)])
for i in range(len(w)):
    print(w[i],'\n',v[:,i])

(-3.8650328730316614+3.241485488980759j) 
 [0.7466503 +0.00000000e+00j 0.15725448-6.46350703e-01j
 0.00385633+5.02292761e-04j]
(-3.8650328730316614-3.241485488980759j) 
 [0.7466503 -0.00000000e+00j 0.15725448+6.46350703e-01j
 0.00385633-5.02292761e-04j]
(-127.77874098424516+0j) 
 [-0.12541357+0.j -0.09486438+0.j  0.9875587 +0.j]


In [203]:
w,v = fast_eigenvalue(x[int(5090)])
for i in range(len(w)):
    print(w[i],'\n',v[:,i])

(-30.975443398425504+5.888543134672153j) 
 [ 0.92494225+0.j         -0.07932327-0.16213152j  0.10951597+0.31608427j]
(-30.975443398425504-5.888543134672153j) 
 [ 0.92494225-0.j         -0.07932327+0.16213152j  0.10951597-0.31608427j]
(1.75544464859462+0j) 
 [ 0.07800718+0.j  0.98757056+0.j -0.13645245+0.j]


In [190]:
def fast_det(value):

    #parameter
    theta,eta,s0,alpha,bb,mu,rho,w,tv = [10,10,10**-6,0.01,-0.5,25,0.1,0.5,0.1]
       #theta,eta,s0,alpha,bb,mu,rho,w,tv = [10,10,0,0.01,-0.5,25,0.1,0.5,0.1]

    #value
    W,L,p,n,s = value
    
    Mat = np.zeros((3,3))
    A = n*p / w
    lambda_ = alpha*(s+s0)**bb
    

    Mat[0,0] = -theta*lambda_
    Mat[0,1] = theta*(lambda_-1)*n/w
    Mat[0,2] = theta*(A-L)

    dot_L = theta*(lambda_*(A-L)-A)
    dot_p = ((1-n)*rho*(mu-p)+eta*(w-W)*(E-A+L)+w*dot_L) / (1-n*w-(1-n)*W)

    #df3dW = -eta*(E-A+L) / (1-w*n-(1-n)*W) - (1-n)*dot_p / (1-n*w-(1-n)*W)**2
    df3dW = -eta*(E-A+L) / (1-w*n-(1-n)*W) + (1-n)*dot_p / (1-n*w-(1-n)*W)**2 # change sign
    df3dl = ( (w-W)*eta + w*Mat[1,1] ) / (1-w*n-(1-n)*W)
    df3dp =  ( (1-n)*(-rho)-(w-W)*eta*n/w + w*Mat[0,1] ) / (1-w*n-(1-n)*W)
    df3dlambda = w*(theta*(A-L)) / (1-w*n-(1-n)*W)

    Mat[1,0] = df3dl
    Mat[1,1] = df3dp
    Mat[1,2] = df3dlambda

    
    df4dl = w/p*(eta+Mat[0,0])
    df4dp = -w/p**2*(eta*(E-A+L)+theta*(lambda_*(A-L)-A)) + w/p * (eta*(-n/w)+Mat[0,1])
    df4dlambda = w/p*theta*(A-L)

    a0 = ((1-n)*rho*(mu-p)*eta*(w-W)*(E-A+L)+w*theta*(-A)) / (p*(1-n*w-(1-n)*W))
    a1 = w*theta*(A-L) / (p*(1-n*w-(1-n)*W))



    #Mat[4,0] = delta / (2*alpha**2) * tv**2 *lambda_**2*2*dot_p/p * df3dW /p
    #Mat[4,1] = delta / (2*alpha**2) * tv**2 *lambda_**2*2*dot_p/p * df3dl /p
    #Mat[4,2] = delta / (2*alpha**2) * tv**2 *lambda_**2*2*dot_p/p * (df3dp /p - dot_p/p**2)
    #Mat[4,3] = delta / (2*alpha**2) * tv**2 *lambda_**2*2*dot_p/p * (df3dn /p)
    #Mat[4,4] = delta / (2*alpha**2) * (alpha**2 - 3*s0*lambda_**2 + tv**2*lambda_**2*(3*a0**2+2*4*a0*a1*lambda_+5*a1**2*lambda_**2))

    
    Mat[2,0] = - delta / (2*alpha**2) * tv**2 *lambda_**3*2*dot_p/p * df3dl /p
    Mat[2,1] = - delta / (2*alpha**2) * tv**2 *lambda_**3*2*dot_p/p * (df3dp /p - dot_p/p**2)
    Mat[2,2] = delta / (2*alpha**2) * (alpha**2 - 3*s0*lambda_**2 - tv**2*lambda_**2*(3*a0**2+2*4*a0*a1*lambda_+5*a1**2*lambda_**2))

    return(np.linalg.det(Mat))

In [191]:
list_det = np.array([fast_det(value) for value in x])/10**7

In [192]:
fig = plt.figure(figsize=(8,8))

fig.subplots_adjust(wspace = 0.5, hspace = 0.3)
ax1 = fig.add_subplot(3,1,1)
ax2 = fig.add_subplot(3,1,2)
ax3 = fig.add_subplot(3,1,3)

T1,T2 = 3640,4700

ax1.plot(t[T1:T2],lambda_[T1:T2], 'g-',label ='s',markersize=0.1)
ax2.plot(t[T1:T2],0*list_det[T1:T2],'black',markersize=1,alpha=0.5)
ax2.plot(t[T1:T2],list_det[T1:T2],'+--',markersize=1,alpha=0.8)
